# OpenAI APIs - Completions

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial covers the following popular APIs:

- `chat/completions`
- `completions`
- `batches`

Check out other tutorials to learn about [vision APIs](https://docs.sglang.ai/backend/openai_api_vision.html) for vision-language models and [embedding APIs](https://docs.sglang.ai/backend/openai_api_embeddings.html) for embedding models.

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")
print(f"Server started on http://localhost:{port}")

[2025-02-23 08:37:15] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='0.0.0.0', port=33733, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=733937569, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable

[2025-02-23 08:37:35 TP0] Init torch distributed begin.
[2025-02-23 08:37:36 TP0] Load weight begin. avail mem=78.30 GB


[2025-02-23 08:37:37 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.13it/s]


...

[2025-02-23 08:37:42 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-02-23 08:37:42] INFO:     Started server process [1039644]
[2025-02-23 08:37:42] INFO:     Waiting for application startup.
[2025-02-23 08:37:42] INFO:     Application startup complete.
[2025-02-23 08:37:42] INFO:     Uvicorn running on http://0.0.0.0:33733 (Press CTRL+C to quit)


[2025-02-23 08:37:42] INFO:     127.0.0.1:57028 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-23 08:37:43] INFO:     127.0.0.1:57036 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-23 08:37:43 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-23 08:37:45] INFO:     127.0.0.1:57052 - "POST /generate HTTP/1.1" 200 OK
[2025-02-23 08:37:45] The server is fired up and ready to roll!


Server started on http://localhost:33733


## Chat Completions

### Usage

The server fully implements the OpenAI API.
It will automatically apply the chat template specified in the Hugging Face tokenizer, if one is available.
You can also specify a custom chat template with `--chat-template` when launching the server.

In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-02-23 08:37:48 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, cache hit rate: 2.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-23 08:37:48 TP0] Decode batch. #running-req: 1, #token: 76, token usage: 0.00, gen throughput (token/s): 6.42, #queue-req: 0
[2025-02-23 08:37:48] INFO:     127.0.0.1:57066 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=128,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(response.choices[0].message.content)

[2025-02-23 08:37:48 TP0] Prefill batch. #new-seq: 1, #new-token: 51, #cached-token: 25, cache hit rate: 20.63%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-23 08:37:48 TP0] frequency_penalty, presence_penalty, and repetition_penalty are not supported when using the default overlap scheduler. They will be ignored. Please add `--disable-overlap` when launching the server if you need these features. The speed will be slower in that case.


[2025-02-23 08:37:49 TP0] Decode batch. #running-req: 1, #token: 106, token usage: 0.01, gen throughput (token/s): 71.24, #queue-req: 0


[2025-02-23 08:37:49 TP0] Decode batch. #running-req: 1, #token: 146, token usage: 0.01, gen throughput (token/s): 68.95, #queue-req: 0


[2025-02-23 08:37:50 TP0] Decode batch. #running-req: 1, #token: 186, token usage: 0.01, gen throughput (token/s): 61.42, #queue-req: 0


[2025-02-23 08:37:50] INFO:     127.0.0.1:57066 - "POST /v1/chat/completions HTTP/1.1" 200 OK


Streaming mode is also supported.

In [4]:
stream = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

[2025-02-23 08:37:50] INFO:     127.0.0.1:57066 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-02-23 08:37:50 TP0] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 30, cache hit rate: 33.73%, token usage: 0.00, #running-req: 0, #queue-req: 0
It looks like you're checking to see if I'm functioning properly. How

 can

 I assist you today?[2025-02-23 08:37:50 TP0] Decode batch. #running-req: 1, #token: 1, token usage: 0.00, gen throughput (token/s): 67.34, #queue-req: 0


## Completions

### Usage
Completions API is similar to Chat Completions API, but without the `messages` parameter or chat templates.

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2025-02-23 08:37:50 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 32.57%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-23 08:37:51 TP0] Decode batch. #running-req: 1, #token: 49, token usage: 0.00, gen throughput (token/s): 79.29, #queue-req: 0


[2025-02-23 08:37:51] INFO:     127.0.0.1:57066 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [6]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2025-02-23 08:37:51 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 31.35%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-23 08:37:51 TP0] frequency_penalty, presence_penalty, and repetition_penalty are not supported when using the default overlap scheduler. They will be ignored. Please add `--disable-overlap` when launching the server if you need these features. The speed will be slower in that case.
[2025-02-23 08:37:51 TP0] Decode batch. #running-req: 1, #token: 26, token usage: 0.00, gen throughput (token/s): 87.18, #queue-req: 0


[2025-02-23 08:37:52 TP0] Decode batch. #running-req: 1, #token: 66, token usage: 0.00, gen throughput (token/s): 87.48, #queue-req: 0


[2025-02-23 08:37:52 TP0] Decode batch. #running-req: 1, #token: 106, token usage: 0.01, gen throughput (token/s): 89.68, #queue-req: 0


[2025-02-23 08:37:53 TP0] Decode batch. #running-req: 1, #token: 146, token usage: 0.01, gen throughput (token/s): 89.36, #queue-req: 0
[2025-02-23 08:37:53] INFO:     127.0.0.1:57066 - "POST /v1/completions HTTP/1.1" 200 OK


## Structured Outputs (JSON, Regex, EBNF)

For OpenAI compatible structed outputs API, refer to [Structured Outputs](https://docs.sglang.ai/backend/structured_outputs.html#OpenAI-Compatible-API) for more details.


## Batches

Batches API for chat completions and completions are also supported. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [7]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2025-02-23 08:37:53] INFO:     127.0.0.1:59554 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-23 08:37:53] INFO:     127.0.0.1:59554 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-23 08:37:53 TP0] Prefill batch. #new-seq: 2, #new-token: 20, #cached-token: 60, cache hit rate: 44.53%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [8]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2025-02-23 08:37:53 TP0] Decode batch. #running-req: 1, #token: 65, token usage: 0.00, gen throughput (token/s): 72.35, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2025-02-23 08:37:56] INFO:     127.0.0.1:59554 - "GET /v1/batches/batch_7ec24518-5302-4895-9d76-d0eca9195486 HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2025-02-23 08:37:56] INFO:     127.0.0.1:59554 - "GET /v1/files/backend_result_file-8ed91009-ddc9-42d0-a83a-a219e4952a9e/content HTTP/1.1" 200 OK


[2025-02-23 08:37:56] INFO:     127.0.0.1:59554 - "DELETE /v1/files/backend_result_file-8ed91009-ddc9-42d0-a83a-a219e4952a9e HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(20):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 64,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2025-02-23 08:37:56] INFO:     127.0.0.1:59566 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-23 08:37:56] INFO:     127.0.0.1:59566 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-23 08:37:56 TP0] Prefill batch. #new-seq: 9, #new-token: 270, #cached-token: 225, cache hit rate: 45.13%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-02-23 08:37:56 TP0] Prefill batch. #new-seq: 11, #new-token: 330, #cached-token: 275, cache hit rate: 45.27%, token usage: 0.01, #running-req: 9, #queue-req: 0


[2025-02-23 08:37:56 TP0] Decode batch. #running-req: 20, #token: 945, token usage: 0.05, gen throughput (token/s): 113.72, #queue-req: 0


[2025-02-23 08:37:57 TP0] Decode batch. #running-req: 20, #token: 1745, token usage: 0.09, gen throughput (token/s): 1632.64, #queue-req: 0


[2025-02-23 08:38:06] INFO:     127.0.0.1:40494 - "GET /v1/batches/batch_2ea5aa18-f18e-4668-b630-5be3744161a9 HTTP/1.1" 200 OK


[2025-02-23 08:38:09] INFO:     127.0.0.1:40494 - "GET /v1/batches/batch_2ea5aa18-f18e-4668-b630-5be3744161a9 HTTP/1.1" 200 OK


[2025-02-23 08:38:12] INFO:     127.0.0.1:40494 - "GET /v1/batches/batch_2ea5aa18-f18e-4668-b630-5be3744161a9 HTTP/1.1" 200 OK


[2025-02-23 08:38:15] INFO:     127.0.0.1:40494 - "GET /v1/batches/batch_2ea5aa18-f18e-4668-b630-5be3744161a9 HTTP/1.1" 200 OK


[2025-02-23 08:38:18] INFO:     127.0.0.1:40494 - "GET /v1/batches/batch_2ea5aa18-f18e-4668-b630-5be3744161a9 HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [10]:
import json
import time
from openai import OpenAI
import os

client = OpenAI(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

requests = []
for i in range(5000):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 128,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
        if os.path.exists(input_file_path):
            os.remove(input_file_path)
            print_highlight("Successfully deleted local batch_requests.jsonl file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2025-02-23 08:38:21] INFO:     127.0.0.1:46096 - "POST /v1/files HTTP/1.1" 200 OK
[2025-02-23 08:38:21] INFO:     127.0.0.1:46096 - "POST /v1/batches HTTP/1.1" 200 OK


[2025-02-23 08:38:22 TP0] Prefill batch. #new-seq: 129, #new-token: 3290, #cached-token: 3805, cache hit rate: 52.28%, token usage: 0.03, #running-req: 0, #queue-req: 22


[2025-02-23 08:38:23 TP0] Decode batch. #running-req: 129, #token: 8023, token usage: 0.39, gen throughput (token/s): 157.79, #queue-req: 4871


[2025-02-23 08:38:24 TP0] Decode batch. #running-req: 129, #token: 13183, token usage: 0.64, gen throughput (token/s): 7461.64, #queue-req: 4871


[2025-02-23 08:38:24 TP0] Decode batch. #running-req: 129, #token: 18343, token usage: 0.90, gen throughput (token/s): 9374.78, #queue-req: 4871


...

[2025-02-23 08:38:30 TP0] Decode batch. #running-req: 129, #token: 129, token usage: 0.01, gen throughput (token/s): 10169.25, #queue-req: 4484
[2025-02-23 08:38:30 TP0] Prefill batch. #new-seq: 129, #new-token: 3870, #cached-token: 3225, cache hit rate: 47.02%, token usage: 0.00, #running-req: 0, #queue-req: 4355


[2025-02-23 08:38:30 TP0] Decode batch. #running-req: 129, #token: 9055, token usage: 0.44, gen throughput (token/s): 8362.24, #queue-req: 4355


[2025-02-23 08:38:31 TP0] Decode batch. #running-req: 129, #token: 14215, token usage: 0.69, gen throughput (token/s): 7369.91, #queue-req: 4355
[2025-02-23 08:38:31] INFO:     127.0.0.1:58586 - "POST /v1/batches/batch_54dd0c4c-76e1-4822-89f5-970439ad2e90/cancel HTTP/1.1" 200 OK


[2025-02-23 08:38:34] INFO:     127.0.0.1:58586 - "GET /v1/batches/batch_54dd0c4c-76e1-4822-89f5-970439ad2e90 HTTP/1.1" 200 OK


[2025-02-23 08:38:34] INFO:     127.0.0.1:58586 - "DELETE /v1/files/backend_input_file-d2b7ee12-c998-4ef8-94d2-06248aa38a69 HTTP/1.1" 200 OK


In [11]:
terminate_process(server_process)